In [1]:
import requests
from bs4 import BeautifulSoup
import re


def getHTMLText(url, code="utf-8"):
    try:
        r = requests.get(url)
        r.raise_for_status()
        r.encoding = code 
        return r.text
    except:
        return ""


def getStockList(lst, stockURL):
    html = getHTMLText(stockURL, "GB2312") 
    soup = BeautifulSoup(html, 'html.parser')
    a = soup.find_all('a')[:200] 
    for i in a:
        try:
            href = i.attrs['href']
            lst.append(re.findall(r"\d{6}", href)[0])
        except:
            continue


def getStockInfo(lst, stockURL, fpath):
    count = 0
    for stock in lst:
        url = stockURL + stock  
        html = getHTMLText(url)
        try:
            if html == "":
                continue
            infoDict = {} 
            soup = BeautifulSoup(html, 'html.parser')
            stockInfo = soup.find('div', attrs={'class': 'stock-info'})
            name = stockInfo.find_all(attrs={'class': 'stock-name'})[0]
            infoDict.update({'股票名称': name.text.split()[0]})  
            keyList = stockInfo.find_all('dt')[:6]  
            valueList = stockInfo.find_all('dd')[:6]
            for i in range(len(keyList)):
                key = keyList[i].text
                val = valueList[i].text
                infoDict[key] = val
            with open(fpath, 'a', encoding='utf-8') as f:
                f.write(str(infoDict) + '\n')
                count = count + 1
                print("\rprogress: {:.2f}%".format(count * 100 / len(lst)), end="")                 
        except:
            count = count + 1
            print("\rprogrss: {:.2f}%".format(count * 100 / len(lst)), end="")
            continue


def main():
    stock_list_url = 'https://quote.eastmoney.com/stock_list.html'  
    stock_info_url = 'https://www.laohu8.com/stock/'  
    output_file = 'Stockv200.csv'
    slist = []
    getStockList(slist, stock_list_url)
    getStockInfo(slist, stock_info_url, output_file)


main()

progress: 64.57%